<a href="https://colab.research.google.com/github/lllljjjjllllqqqq/123/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!ls "/content/drive/My Drive/tianchi_demo/"

chinese_roberta_wwm_ext_L-12_H-768_A-12.zip	    subs
chinese_roberta_wwm_large_ext_L-24_H-1024_A-16.zip  TNEWS_train1128.csv
OCEMOTION_train1128.csv				    Untitled0.ipynb
OCNLI_train1128.csv


In [4]:
!nvidia-smi

Sun Feb 21 15:20:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=931d14cb5111d3a39df96b08a661767976a86b33b86b872b4601b59d51508ee2
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [8]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/tianchi_demo/chinese_roberta_wwm_ext_L-12_H-768_A-12.zip','r') as zzz:
    print(zzz.namelist())
    for x in zzz.namelist():
      zzz.extract(x)

['bert_config.json', 'bert_model.ckpt.data-00000-of-00001', 'bert_model.ckpt.index', 'bert_model.ckpt.meta', 'vocab.txt']


In [ ]:
file_name = wget.download('http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531841/OCEMOTION_train1128.csv')
print(file_name)

OCEMOTION_train1128.csv


In [9]:
!ls

bert_config.json		     bert_model.ckpt.meta  vocab.txt
bert_model.ckpt.data-00000-of-00001  drive
bert_model.ckpt.index		     sample_data


In [10]:
pip install keras-bert #keras tensorflow pycaret fasttext

  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=b3a0886b42d4456a65647aca66ef3f782d8c645218ff9d8db9e0e9953cb0be55
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12944 sha256=61ee445f41375555383d7e65272ac7b987198b0a0ba8da09e51187cc3fc7df1d
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=91e156e3d6cc6cc3d750550578d5041e814694f2b101d3e5b9c8a36eecf63be2
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=af76f7ae6b018d7256260632e6127597752943ae3a354689608579154b4d0c4a
  Stor

In [11]:
import pandas as pd
import numpy as np
import os, sys
import codecs, gc
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras.metrics import top_k_categorical_accuracy
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [12]:
#路径配置
path = '..'
docker_data_path = '/tcdata/'

'''
路径说明：
|--tianchi-nlp
​	 |-- code
	|-- data
	|-- subs
  |-- chinese_roberta_wwm_large_ext_L-24_H-1024_A-16

其中：
/code 保存代码
/data #保存数据
/subs #保存数据
/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16 #bert路径
'''




#一些调优参数
er_patience = 2 #early_stopping patience
lr_patience = 5 #ReduceLROnPlateau patience
max_epochs = 3 #epochs
lr_rate = 3e-5 #learning rate
batch_sz =16  #batch_size 4
maxlen = 200  #设置序列长度为，base模型要保证序列长度不超过512
lr_factor = 0.85 #ReduceLROnPlateau factor，lr*=factor
maxlentext1 = 200 #选择ocnli子句一的长度
n_folds = 10   #设置验证集的占比: 1/n_folds
#ReduceLROnPlateau其他参数：https://keras.io/zh/callbacks/#reducelronplateau

#path_bert = path + '/chinese_rbt6_L-6_H-768_A-12/'
#预训练好的模型相关路径
config_path ='/content/bert_config.json'
dict_path   ='/content/vocab.txt'
checkpoint_path ='/content/bert_model.ckpt'

In [13]:
#数据读取
path='/content/drive/MyDrive/tianchi_demo/'
#将ocnli中content1[0:maxlentext1]+content2作为ocnli的content输入
times_train = pd.read_csv(path + 'TNEWS_train1128.csv',    sep='\t', header=None, names=('id', 'content', 'label')).astype(str)
ocemo_train = pd.read_csv(path + 'OCEMOTION_train1128.csv',sep='\t', header=None, names=('id', 'content', 'label')).astype(str)
ocnli_train = pd.read_csv(path + 'OCNLI_train1128.csv',    sep='\t', header=None, names=('id', 'content1', 'content2', 'label')).astype(str)
ocnli_train['content'] = ocnli_train['content1'] + ocnli_train['content2']#.apply( lambda x: x[:maxlentext1] )


In [20]:
#EDA分析
#子句文本长度统计分析
times_train['content'].str.len().describe(percentiles=[.99, .999, .9999]),ocemo_train['content'].str.len().describe(percentiles=[.99, .999, .9999])\
,ocnli_train['content1'].str.len().describe(percentiles=[.99, .999, .9999]),ocnli_train['content2'].str.len().describe(percentiles=[.99, .999, .9999])

(count     63360.000000
 mean         22.171086
 std           7.334206
 min           2.000000
 50%          22.000000
 99%          39.000000
 99.9%        48.000000
 99.99%       58.000000
 max         145.000000
 Name: content, dtype: float64, count     35315.000000
 mean         48.214328
 std          84.391942
 min           3.000000
 50%          34.000000
 99%         142.000000
 99.9%       172.000000
 99.99%     1040.953200
 max       12326.000000
 Name: content, dtype: float64, count     48778.000000
 mean         24.174607
 std          11.515428
 min           8.000000
 50%          22.000000
 99%          50.000000
 99.9%        50.000000
 99.99%       50.000000
 max          50.000000
 Name: content1, dtype: float64, count      48778.000000
 mean          15.828529
 std          977.396848
 min            2.000000
 50%           10.000000
 99%           27.000000
 99.9%         40.000000
 99.99%        54.000000
 max       215874.000000
 Name: content2, dtype: float64)

In [14]:
#构造输入、标签数据
#合并三个任务的训练、测试数据，将三任务转为单个任务
train_df = pd.concat([times_train, ocemo_train, ocnli_train[['id','content', 'label']]], axis=0).copy()
#LabelEncoder处理标签，因为label需要从0开始
encode_label = LabelEncoder()
train_df['label'] = encode_label.fit_transform(train_df['label'].apply(str))

###采用分层抽样的方式，从训练集中抽取10%作为验证集
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=222)

X_trn = pd.DataFrame()
X_val = pd.DataFrame()

for train_index, test_index in skf.split(train_df.copy(), train_df['label']):
  X_trn, X_val = train_df.iloc[train_index], train_df.iloc[test_index]
  break
###



#训练数据、测试数据和标签转化为模型输入格式
n_cls = len( train_df['label'].unique() )

#训练集每行的content、label转为tuple存入list，再转为numpy array
TRN_LIST = []
for data_row in X_trn.iloc[:].itertuples():
  TRN_LIST.append((data_row.content, to_categorical(data_row.label, n_cls)))
TRN_LIST = np.array(TRN_LIST)

#验证集每行的content、label转为tuple存入list，再转为numpy array
VAL_LIST = []
for data_row in X_val.iloc[:].itertuples():
  VAL_LIST.append((data_row.content, to_categorical(data_row.label, n_cls)))
VAL_LIST = np.array(VAL_LIST)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
#EDA分析
#标签占比统计分析
train_df['label'].value_counts() / train_df.shape[0]

1     0.113467
0     0.109940
17    0.107397
23    0.084603
21    0.060318
10    0.047771
6     0.041749
4     0.039918
13    0.039036
8     0.033292
3     0.032668
5     0.032268
11    0.029487
19    0.029481
9     0.027690
18    0.027588
12    0.027541
16    0.027460
22    0.027412
15    0.022923
7     0.016853
2     0.008993
24    0.006097
20    0.004001
14    0.002048
Name: label, dtype: float64

In [16]:
#数据预处理
#让每条文本的长度相同，用0填充
def seq_padding(X, padding=0):
  L = [len(x) for x in X]
  ML = max(L)
  return np.array([  np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X  ])

#将词表中的词编号转换为字典
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
  for line in reader:
    token = line.strip()
    token_dict[token] = len(token_dict)

#重写tokenizer        
class OurTokenizer(Tokenizer):
  def _tokenize(self, text):
    R = []
    for c in text:
      if c in self._token_dict:
        R.append(c)
      elif self._is_space(c):
        R.append('[unused1]') # 用[unused1]来表示空格类字符
      else:
        R.append('[UNK]')     # 不在列表的字符用[UNK]表示
    return R

tokenizer = OurTokenizer(token_dict)

In [17]:
#data_generator只是一种为了节约内存的数据方式
class data_generator:
  global batch_sz
  def __init__(self, data, batch_size=batch_sz, shuffle=True):#此处修改batch_size
    self.data = data
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.steps = len(self.data) // self.batch_size
    if len(self.data) % self.batch_size != 0:
      self.steps += 1

  def __len__(self):
    return self.steps

  def __iter__(self):
    while True:
      idxs = list(range(len(self.data)))
      if self.shuffle:
        np.random.shuffle(idxs)
      X1, X2, Y = [], [], []
      for i in idxs:
        d = self.data[i]
        text = d[0][: maxlen]
        #text2 = d[0][maxlentext1:maxlen]
        x1, x2 = tokenizer.encode(first=text)#1, second=text2)#修改成输入两个子句

        y = d[1]
        X1.append(x1)
        X2.append(x2)
        Y.append([y])
        if len(X1) == self.batch_size or i == idxs[-1]:
          X1 = seq_padding(X1)
          X2 = seq_padding(X2)
          Y = seq_padding(Y)
          yield [X1, X2], Y[:, 0, :]
          [X1, X2, Y] = [], [], []

In [18]:

#定义计算每个batch的f1
from keras import backend as K

def f1(y_true, y_pred):
  def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

  def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
  precision = precision(y_true, y_pred)
  recall = recall(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [19]:
#bert模型搭建与配置
def build_bert(nclass):
  global lr_rate
  bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)  #加载预训练模型

  for l in bert_model.layers:
    l.trainable = True

  x1_in = Input(shape=(None,))
  x2_in = Input(shape=(None,))

  x = bert_model([x1_in, x2_in])
  x = Lambda(lambda x: x[:, 0])(x) #取出[CLS]对应的向量用来做分类
  p = Dense(nclass, activation='softmax')(x) #接dense层softmax输出

  model = Model([x1_in, x2_in], p)
  model.compile(loss='categorical_crossentropy',#损失函数
                optimizer=Adam(lr_rate),    #优化器选择及学习率
                metrics=['accuracy', f1])   #评价函数

  print(model.summary())
  return model

In [20]:
#交叉验证训练和测试模型
def run_nocv(nfold, trn_data, val_data, data_labels, n_cls):
  global er_patience
  global lr_patience
  global max_epochs
  global f1metrics
  global lr_factor
  #test_model_pred = np.zeros((len(data_test), n_cls))

  model = build_bert(n_cls)
  #下行代码用于加载保存的权重继续训练
  #model.load_weights(path + '/subs/model.epoch02_val_loss1.0146_val_acc0.6479_val_f10.6359.hdf5')
  
  early_stopping = EarlyStopping(monitor= "val_f1", patience=er_patience) #早停法，防止过拟合 #'val_accuracy'
  plateau = ReduceLROnPlateau(monitor="val_f1", verbose=1, mode='max', factor=lr_factor, patience=lr_patience) #当评价指标不在提升时，减少学习率 #"val_accuracy"
  checkpoint = ModelCheckpoint(path + "subs/model.epoch{epoch:02d}_val_loss{val_loss:.4f}_val_acc{val_accuracy:.4f}_val_f1{val_f1:.4f}.hdf5", monitor="val_f1", verbose=2, save_best_only=True, mode='max', save_weights_only=True) #保存val_f1最好的模型权重#'val_acc'

  #训练跟验证集可shuffle打乱，测试集不可打乱也无必要（不然在生成线上结果时没法跟ID对应上）
  #其他
  train_D = data_generator(trn_data, shuffle=True)
  valid_D = data_generator(val_data, shuffle=True)
  #test_D = data_generator(data_test, shuffle=False)
  
  #模型训练
  model.fit_generator(
      train_D.__iter__(),
      steps_per_epoch=len(train_D),
      epochs=max_epochs,
      validation_data=valid_D.__iter__(),
      validation_steps=len(valid_D),
      callbacks=[early_stopping, plateau, checkpoint],
  )
  model.save(path+'bert_model.hdf5')
  #模型预测
  
  test_model_pred = model.predict_generator(train_D.__iter__(), steps=len(test_D), verbose=1)
  train_model_pred = test_model_pred#model.predict(train_D.__iter__(), steps=len(train_D), verbose=1)
 
  del model
  gc.collect()   #清理内存
  K.clear_session() #clear_session就是清除一个session

  return test_model_pred, train_model_pred

In [ ]:
#'''
#此cell代码为训练代码
cvs = 1
#输出为numpy array格式的25列概率
test_model_pred, train_model_pred = run_nocv(cvs, TRN_LIST, VAL_LIST, None, n_cls)

#将结果转为DataFrame格式
preds_tst_df = pd.DataFrame(test_model_pred)
preds_trn_df = pd.DataFrame(train_model_pred)

#再将range(0,25)做encode_label逆变换作为该DataFrame的列名
preds_col_names = encode_label.inverse_transform( range(0,n_cls) )
preds_tst_df.columns = preds_col_names
preds_trn_df.columns = preds_col_names

#保存输出概率，将该输出概率输入树模型还能提升分数，大概4个千分点
#preds_tst_df.to_csv(path + "/subs/tstsprob.csv", index=None)
#preds_trn_df.to_csv(path + "/subs/trnsprob.csv", index=None)
#'''

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, None, 768)    101677056   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 768)          0           model_1[0][0]              

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1908/8295 [=====>........................] - ETA: 50:39 - loss: 1.6125 - accuracy: 0.4554 - f1: 0.3792